In [1]:
import pandas as pd

In [2]:
url = './data/서울특별시 광진구_도로열선 설치 현황_20250131.csv'

In [3]:
data = pd.read_csv(url, encoding='euc-kr')
data.head(2)

,연번,행정동,노선명,기점,종점,열선연장(m) (1차로 기준),차로수,설치년도,설치월,비고
0,1,구의2동,동의초 통학로,구의2동 34-8,구의2동 81-3,278,2,2022.0,11.0,NaN
1,2,구의2동,동의초 통학로,구의2동 23-26,구의2동 23-30,76,2,2022.0,11.0,NaN


In [4]:
data.columns

Index(['연번', '행정동', '노선명', '기점', '종점', '열선연장(m) (1차로 기준)', '차로수', '설치년도',
       '설치월', '비고'],
      dtype='object')

In [5]:
line_start = data['기점'].to_list() 
line_end = data['종점'].to_list()  

### kakao map api

In [6]:
api_key_url = '../API_KEY/API_KEYS.xlsx'
open_api_key_file = pd.read_excel(api_key_url)

In [7]:
open_api_key_file['사이트'].value_counts()

사이트
공공데이터                 1
kakao_api_key_REST    1
카카오 JS                1
국토부v-world            1
GOOGLE_MAP            1
Name: count, dtype: int64

In [8]:
KAKAO_API_KEY = open_api_key_file[open_api_key_file['사이트'] == 'kakao_api_key_REST'].values[0][1]  

In [9]:
import requests

def kakao_api(address):
    address = '서울' + address
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    params = {"query": address}

    response = requests.get(url, headers=headers, params=params)
    result = response.json()
    
    if result.get("documents"):
        location = result["documents"][0]
        lat = float(location["y"])
        lon = float(location["x"])
        return lat, lon
    print(address)
    print("주소를 찾을 수 없음:", result)
    return None, None
 


In [10]:
def get_address_list_before_kakao(addresses):
    latitudes, longitudes = [], []
    for address in addresses:
        lat, lon = kakao_api(address)
        if lat is not None and lon is not None:
            latitudes.append(lat)
            longitudes.append(lon)
    return latitudes, longitudes

get_start_lat, get_start_lon = get_address_list_before_kakao(line_start)
get_end_lat, get_end_lon = get_address_list_before_kakao(line_end)

In [11]:
print(f'출발지의 좌표 길이 : {len(get_start_lat)}, {len(get_start_lon)}')
print(f'도착지의 좌표 길이 : {len(get_end_lat)}, {len(get_end_lon)}')

출발지의 좌표 길이 : 41, 41
도착지의 좌표 길이 : 41, 41


In [12]:
average_lat = []
average_lon = []

for i in range(max(len(get_start_lat), len(get_end_lat))):
    start_lat = get_start_lat[i] if i < len(get_start_lat) else get_start_lat[-1]
    end_lat = get_end_lat[i] if i < len(get_end_lat) else get_end_lat[-1]
    average_lat.append((start_lat + end_lat) / 2)

for i in range(max(len(get_start_lon), len(get_end_lon))):
    start_lon = get_start_lon[i] if i < len(get_start_lon) else get_start_lon[-1]
    end_lon = get_end_lon[i] if i < len(get_end_lon) else get_end_lon[-1]
    average_lon.append((start_lon + end_lon) / 2)


In [13]:
print(f'중간값 좌표 길이 : {len(average_lat)}, {len(average_lon)}')

중간값 좌표 길이 : 41, 41


In [14]:
data['기점_좌표_위도'] = get_start_lat
data['기점_좌표_경도'] = get_start_lon
 
data['종점_좌표_위도'] = get_end_lat
data['종점_좌표_경도'] = get_end_lon 

In [15]:
data['열선_중앙_위도'] = average_lat
data['열선_중앙_경도'] = average_lon

In [16]:
data = data.drop('비고', axis = 1) 
data = data.drop('연번', axis = 1) 
data = data.drop('설치년도', axis = 1) 
data = data.drop('설치월', axis = 1)  

In [17]:
data.columns

Index(['행정동', '노선명', '기점', '종점', '열선연장(m) (1차로 기준)', '차로수', '기점_좌표_위도',
       '기점_좌표_경도', '종점_좌표_위도', '종점_좌표_경도', '열선_중앙_위도', '열선_중앙_경도'],
      dtype='object')

In [18]:
save_file = './make_file/열선_위경도값(중앙값_포함).csv'
data.to_csv(save_file, encoding='euc-kr') 
print(f'{save_file} 저장 완료')

./make_file/열선_위경도값(중앙값_포함).csv 저장 완료
